In [1]:
import sys, os

# Get the absolute path of the parent folder (where config.py lives)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print("Added to sys.path:", parent_dir)

# now your imports should work
from config import get_config
from train_utils.gpu_utils import get_device_summary
from data.loader import get_dataloaders


Added to sys.path: /home/arsalan/wsu-grid/ml-jet-param-predictor


In [2]:
import os
import torch
import torch.nn as nn
import json

from config import get_config
from train_utils.gpu_utils import get_device_summary
from data.loader import get_dataloaders
from models.model import create_model
from train_utils.resume import init_resume_state
from train_utils.resume import fill_trackers_from_history
from train_utils.resume import load_pretrained_model
from train_utils.training_loop import run_training_loop
from train_utils.scheduler_utils import create_scheduler
from train_utils.training_summary import finalize_training_summary
from train_utils.training_summary import print_best_model_summary
from train_utils.plot_metrics import plot_train_val_metrics
from train_utils.plot_metrics import plot_loss_accuracy
from train_utils.plot_metrics import plot_confusion_matrices


In [15]:
# cfg=get_config(config_path="/home/arsalan/wsu-grid/ml-jet-param-predictor/config/convnext_gaussian_bs32_ep50_lr1e-04_ds7200000_g500_sched-RLRP.yml")
# cfg=get_config(config_path="/home/arsalan/wsu-grid/ml-jet-param-predictor/config/efficientnet_bs32_ep50_lr1e-02_ds7200000_g500.yml")

cfg=get_config(config_path="/home/arsalan/wsu-grid/ml-jet-param-predictor/config/swin_bs32_ep50_lr1e-04_ds7200000_g500.yml")
# from models.model_vit import create_model
# cfg=get_config(config_path="/home/arsalan/wsu-grid/ml-jet-param-predictor/" \
# "experiments/exp_preload_trained_model_and_train_more/config/vit_tiny_patch16_224_gaussian_bs32_ep50_lr1e-04_p12_ds7200000_g500_sched-RLRP_preload_p4.yml")

# from models.model_mamba import create_model
# cfg=get_config(config_path="/home/arsalan/wsu-grid/ml-jet-param-predictor/" \
# "experiments/exp_preload_trained_model_and_train_more/config/mambaout_base_plus_rw_bs16_ep50_lr1e-04_p12_ds7200000_g500_sched-RLRP_preload-p4.yml")

# cfg=get_config()
print(json.dumps(vars(cfg), indent=2))

[INFO] Config Path: /home/arsalan/wsu-grid/ml-jet-param-predictor/config/swin_bs32_ep50_lr1e-04_ds7200000_g500.yml
[INFO] Detected native Ubuntu host: DS044955
[INFO] Using dataset root: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/data/jet_ml_benchmark_config_01_to_09_alpha_0.2_0.3_0.4_q0_1.5_2.0_2.5_MMAT_MLBT_size_7200000_balanced_unshuffled
[INFO] Extracted dataset_size from path: 7200000
{
  "model_tag": "Swin_g500",
  "backbone": "swin",
  "batch_size": 32,
  "epochs": 50,
  "learning_rate": 0.0001,
  "patience": 10,
  "input_shape": [
    1,
    32,
    32
  ],
  "global_max": 121.79151153564453,
  "dataset_root_dir": "/home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/data/jet_ml_benchmark_config_01_to_09_alpha_0.2_0.3_0.4_q0_1.5_2.0_2.5_MMAT_MLBT_size_7200000_balanced_unshuffled",
  "train_csv": "/home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/data/jet_ml_benchmark_config_01_to_09_alpha_0.2_0.3_0.4_q0_1.5_2.0_2.5_MMAT_MLBT_size_7200000_balanced_

In [16]:
os.makedirs(cfg.output_dir, exist_ok=True)
print(f"[INFO] Saving all outputs to: {cfg.output_dir}")

[INFO] Saving all outputs to: training_output/Swin_g500_bs32_ep50_lr1e-04_ds7200000_g500_sched_ReduceLROnPlateau


In [17]:
device= get_device_summary()

=== NVIDIA-SMI ===
Fri Sep  5 18:44:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070        Off | 00000000:01:00.0  On |                  N/A |
| 30%   37C    P5              24W / 220W |   1753MiB /  8192MiB |     23%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+------

In [18]:
# Model and optimizer
model, optimizer = create_model(cfg.backbone, cfg.input_shape, cfg.learning_rate)
model.to(device)

MultiHeadClassifier(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (layers): Sequential(
      (0): SwinTransformerStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path1): Identity()
            (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384,

In [ ]:
# # Count trainable parameters
# num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f"Trainable parameters: {num_params:,}")

# # (Optional) Count all parameters including frozen ones
# num_params_total = sum(p.numel() for p in model.parameters())
# print(f"Total parameters: {num_params_total:,}")


Trainable parameters: 5,525,960
Total parameters: 5,525,960


In [19]:
# ===== FLOPs / Params utilities =====
# Drop this in a cell after you've created the model

import torch
import math

def _make_dummy_input(input_shape, device):
    # input_shape: (C, H, W)
    return torch.randn(1, *input_shape, device=device)

def count_trainable_params(model: torch.nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def profile_with_fvcore(model, input_shape, device):
    try:
        from fvcore.nn import FlopCountAnalysis, parameter_count
    except Exception as e:
        return None
    model_eval = model.eval()
    dummy = _make_dummy_input(input_shape, device)
    with torch.no_grad():
        flops = FlopCountAnalysis(model_eval, dummy).total()        # FLOPs (float ops)
    params = parameter_count(model_eval)[""]
    macs = flops / 2.0                                              # approx: 1 MAC ≈ 2 FLOPs
    return dict(params=params, macs=macs, flops=flops)

def profile_with_ptflops(model, input_shape, device):
    try:
        from ptflops import get_model_complexity_info
    except Exception as e:
        return None
    C, H, W = input_shape
    # ptflops expects (C, H, W) and returns MACs
    model_eval = model.eval().to(device)
    with torch.no_grad():
        macs_str, params_str = get_model_complexity_info(
            model_eval, (C, H, W), as_strings=True, print_per_layer_stat=False, verbose=False
        )
    # Parse strings like '12.34 M', '0.56 G'
    def _to_num(s):
        s = s.strip().upper().replace(' ', '')
        if s.endswith('K'): return float(s[:-1]) * 1e3
        if s.endswith('M'): return float(s[:-1]) * 1e6
        if s.endswith('G'): return float(s[:-1]) * 1e9
        return float(s)
    macs = _to_num(macs_str)
    params = _to_num(params_str)
    flops = macs * 2.0  # common convention
    return dict(params=params, macs=macs, flops=flops)

def profile_model(model, input_shape=(1,32,32), device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Returns a dict with:
      - params (int)
      - macs   (float, operations)
      - flops  (float, operations)
      - pretty strings for Params (M), MACs (G), FLOPs (G)
    """
    model = model.to(device).eval()
    # First: fvcore (FLOPs), else: ptflops (MACs)
    result = profile_with_fvcore(model, input_shape, device)
    if result is None:
        result = profile_with_ptflops(model, input_shape, device)
    if result is None:
        raise RuntimeError("Neither fvcore nor ptflops is available. Install one of them to compute FLOPs/MACs.")

    params = int(result["params"])
    macs   = float(result["macs"])
    flops  = float(result["flops"])

    pretty = {
        "Params (M)": f"{params/1e6:.2f}",
        "MACs (G)" : f"{macs/1e9:.3f}",
        "FLOPs (G)": f"{flops/1e9:.3f}",
    }
    return {"params": params, "macs": macs, "flops": flops, "pretty": pretty}




In [20]:
# ===== Example usage after you build your model =====
# model, optimizer = create_model(cfg.backbone, cfg.input_shape, cfg.learning_rate)
# model.to(device)

try:
    prof = profile_model(model, input_shape=cfg.input_shape, device=device)
    print("Trainable parameters:", f"{count_trainable_params(model):,}")
    print("All parameters:", f"{sum(p.numel() for p in model.parameters()):,}")
    print("== Model profile ==")
    for k, v in prof["pretty"].items():
        print(f"{k}: {v}")
except RuntimeError as e:
    print(e)
    print("\nQuick install (choose one):")
    print("  !pip install fvcore")
    print("  # or")
    print("  !pip install ptflops")

Unsupported operator aten::rsub encountered 24 time(s)
Unsupported operator aten::pad encountered 15 time(s)
Unsupported operator aten::mul encountered 12 time(s)
Unsupported operator aten::add encountered 37 time(s)
Unsupported operator aten::softmax encountered 12 time(s)
Unsupported operator aten::gelu encountered 12 time(s)
Unsupported operator aten::mean encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
backbone.layers.0.blocks.1.drop_path1, backbone.layers.0.blocks.1.drop_path2, backbone.layers.1.blocks.0.drop_path1, backbone.layers.1.blocks.0.drop_path2, backbone.layers.1.blocks.1.drop_path1, backbone.layers.1.blocks.1.drop_path2, backbone.layers.2.blocks.0.drop_path1, backbone.layers.2.blo

Trainable parameters: 27,500,690
All parameters: 27,500,690
== Model profile ==
Params (M): 27.50
MACs (G): 0.045
FLOPs (G): 0.090
